In [1]:
INDEX_FILE = '../index.json'

In [2]:
DATASET_DIR = '/Volumes/Manan/HandGestureDataset_SHREC2017'

## Prelude

In [3]:
from typing import Tuple
from pathlib import Path
from configparser import ConfigParser
import json

In [4]:
labels= [
    None,
    "Grab",
    "Tap",
    "Expand",
    "Pinch",
    "Rotation CW",
    "Rotation CCW",
    "Swipe Right",
    "Swipe Left",
    "Swipe Up",
    "Swipe Down",
    "Swipe X",
    "Swipe V",
    "Swipe +",
    "Shake"
]

## Walkers

In [5]:
def getpcds(d: Path) -> list:
    files = d.iterdir()
    files = filter(lambda f: f.suffix == '.pcd', files)
    files = sorted(files, key=lambda f: int(f.name.split('_')[0]))
    files = map(lambda f: f.parts[-5:], files)
    files = map(lambda ps: '/data/' + '/'.join(ps), files)
    files = list(files)
    return files

In [6]:
getdirs = lambda d: filter(lambda e: e.is_dir(), d.iterdir())

In [7]:
def walk_dataset(p: str) -> dict:
    p = Path(p)
    D = dict()
    
    # gestures
    for g in getdirs(p):
        gid = int(g.name.split('_').pop(1))
        essais = list()
        D[g.name] = {
            "id": gid,
            "label": labels[gid],
            "essais": essais
        }
        
        # fingers
        for f in getdirs(g):
            D[g.name][f.name] = dict()
            
            # subjects
            for s in getdirs(f):
                D[g.name][f.name][s.name] = dict()
                
                # essais
                for e in getdirs(s):
                    pcds = getpcds(e)
                    D[g.name][f.name][s.name][e.name] = pcds
                    essais.append('/data/' + '/'.join(e.parts[-4:]))
                # !essais
            # !subjects
        #!fingers
    #!gestures
    
    
    return D

## Configuration and Run

In [8]:
x = walk_dataset(DATASET_DIR)

In [9]:
with open(INDEX_FILE, 'w') as f:
    json.dump(x, f)